In [1]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install apscheduler

     ---------------------------------------- 59.3/59.3 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install oandapyV20

     ---------------------------------------- 51.6/51.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for oandapyV20: started
  Running setup.py install for oandapyV20: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: oandapyV20 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install oanda_candles

     ---------------------------------------- 52.7/52.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 99.9/99.9 kB ? eta 0:00:00
     ---------------------------------------- 66.8/66.8 kB ? eta 0:00:00
     -------------------------------------- 121.1/121.1 kB 7.4 MB/s eta 0:00:00
     -------------------------------------- 163.8/163.8 kB 9.6 MB/s eta 0:00:00
     ------------------------------------- 248.1/248.1 kB 14.9 MB/s eta 0:00:00
     ---------------------------------------- 98.7/98.7 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: oandapyV20
    Found existing installation: oandapyV20 0.7.2
    U

  DEPRECATION: oandapyV20 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  DEPRECATION: atomicwrites is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] 

In [5]:
import pandas as pd
import pandas_ta as ta
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [6]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    #Get range of candles in window
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    #Check if all EMAA_fast vals are < EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

In [7]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles) == 2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #anddf.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles) == 1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
          return 1
    return 0

Connecting to Market

In [8]:
access_token = ''
accountID = '101-001-25962876-001'

In [10]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M5)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)
    print(candle)

True
<Candle: 2024-05-01 09:45 AM>
True
<Candle: 2024-05-01 09:50 AM>
True
<Candle: 2024-05-01 09:55 AM>


In [11]:
def count_opened_trades():
    client = API(access_token=access_token)
    r = trades.OpenTrades(accountID=accountID)
    client.request(r)
    return len(r.response['trades'])

In [12]:
def get_candles_frame(n):
    candles = get_candles(n)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] =float(str(candle.bid.l))
        i = i + 1
    
    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['CLose'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    dfstream["ATR"] = ta.atr(dfstream.High, dfstream.Low, dfstream.Close, length=7)
    dfstream["EMA_fast"] = ta.ema(dfstream.Close, length=30)
    dfstream["EMA_slow"] = ta.ema(dfstream.Close, length=50)
    dfstream['RSI'] = ta.rsi(dfstream.Close, length=10)
    my_bbands = ta.bbands(dfstream.Close, length=15, std=1.5)
    dfstream = dfstream.join(my_bbands)

    return dfstream